In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import os
import random
import json
#import spacy
import nltk
from tqdm import tqdm
import re
from vqa import VQA

In [2]:
%matplotlib inline
SEED = 2137
random.seed(SEED)
np.random.seed(SEED)

In [3]:
vqa = VQA(annotation_file= "data/v2_mscoco_train2014_annotations.json", question_file="data/v2_OpenEnded_mscoco_train2014_questions.json")
vqa_val = VQA(annotation_file= "data/v2_mscoco_val2014_annotations.json", question_file="data/v2_OpenEnded_mscoco_val2014_questions.json")

q_ids_train = vqa.getImgIds()
#q_ids_val = vqa_val.getImgIds()
#q_ids_all = q_ids_train+q_ids_val

loading VQA annotations and questions into memory...
0:00:05.496582
creating index...
index created!
loading VQA annotations and questions into memory...
0:00:03.354859
creating index...
index created!


## VQA tool

In [4]:
ids = vqa.getImgIds()
annIds = vqa.getQuesIds(imgIds=random.sample(ids,5))
anns = vqa.loadQA(annIds)
#randomAnn = random.choice(anns)
#vqa.showQA([randomAnn])
#imgId = randomAnn['image_id']

In [5]:
# question = "What can be seen in this image?"

# "annotations": [{"question_type": "describe image", "multiple_choice_answer": "down", "answers": [], "image_id": 262148, "answer_type": "caption", "question_id": question_id}]
# annotation{
# "question_id" : int,
# "image_id" : int,
# "question_type" : str,
# "answer_type" : str,
# "answers" : [answer],
# "multiple_choice_answer" : str
# }

In [6]:
all_ids = vqa.getImgIds()

In [7]:
len(all_ids)

443757

In [8]:
all_anns = vqa.dataset["annotations"]
my_qas = [{'image_id':ann["image_id"],
           'question':vqa.qqa[ann["question_id"]]["question"],
           'question_type':ann["question_type"],
           'answer':ann["multiple_choice_answer"],
           'answer_type':ann["answer_type"]}
           for ann in all_anns]

In [9]:
dic = dict()
for qa in my_qas:
    val = dic.get((qa["answer_type"], qa["question_type"]), [])
    val.append((qa["question"], qa["answer"]))
    dic[(qa["answer_type"], qa["question_type"])] = val
dic = dict(sorted(dic.items()))
print(len(dic))
for k in dic:
    vals = dic[k]
    lv = len(vals)
    print('Key =', k, 'nr vals =', lv)
    limit = lv if lv <= 10 else 3
    for i, val in enumerate(vals):
        print(val)
        if i == limit:
            break

143
Key = ('number', 'are') nr vals = 10
('Are many people playing tennis?', 'yes')
('Are those for one or two devices?', '1')
('Are many people waiting?', '0')
('Are many types of personnel needed before this type of vehicle can take off?', 'yes')
('Are many people playing tennis?', '0')
('Are many people watching the match?', '4')
('Are How many trees in the photo?', '3')
('Are many people waiting?', 'yes')
('Are How many trees in the photo?', '10')
('Are those for one or two devices?', '2')
Key = ('number', 'are the') nr vals = 2
('Are the sheep two legs mammals or four legs mammals?', '4')
('Are the sheep two legs mammals or four legs mammals?', '4')
Key = ('number', 'are there') nr vals = 17
('Are there 1 or 2 ovens?', '2')
('Are there many people in the room?', '1')
('Are there one or two buses?', '2')
('Are there two cats or just one looking in a mirror?', '1')
Key = ('number', 'are there any') nr vals = 1
('Are there any people in the image?', '1')
Key = ('number', 'are they') 

In [10]:
all_anns_val = vqa_val.dataset["annotations"]
my_qas_val = [{'image_id':ann["image_id"],
           'question':vqa_val.qqa[ann["question_id"]]["question"],
           'question_type':ann["question_type"],
           'answer':ann["multiple_choice_answer"],
           'answer_type':ann["answer_type"]}
           for ann in all_anns_val]

In [11]:
dic_val = dict()
for qa in my_qas_val:
    val = dic_val.get((qa["answer_type"], qa["question_type"]), [])
    val.append((qa["question"], qa["answer"]))
    dic_val[(qa["answer_type"], qa["question_type"])] = val
dic_val = dict(sorted(dic_val.items()))
print(len(dic_val))
for k, vals in dic_val.items():
    lv = len(vals)
    print('Key =', k, 'nr vals =', lv)
    limit = lv if lv <= 10 else 3
    for i, val in enumerate(vals):
        print(val)
        if i == limit:
            break

143
Key = ('number', 'are the') nr vals = 2
('Are the computer monitors hooked up to just one computer or two?', '2')
('Are the computer monitors hooked up to just one computer or two?', '1')
Key = ('number', 'are there') nr vals = 3
('Are there 4 lights or 8 lights?', '4')
('Are there two planes or is it a reflection?', '2 planes')
('Are there one or two men in the picture?', '1')
Key = ('number', 'are these') nr vals = 2
('Are these parts to 1 or more than 1 remote?', 'more than 1')
('Are these parts to 1 or more than 1 remote?', '1')
Key = ('number', 'can you') nr vals = 2
("Can you tell me this man's age?", 'no')
('Can you type the number of the train?', '753')
Key = ('number', 'does the') nr vals = 8
('Does the bus have one or two sections?', '2')
('Does the refrigerator have one or two doors?', '2')
('Does the refrigerator have one or two doors?', '1')
('Does the kite have 3, 4, or 5 sides?', '4')
('Does the bus have one or two sections?', '2')
('Does the traffic on the street fl

In [12]:
keys1 = dic.keys()
keys2 = dic_val.keys()

In [13]:
difference12 = keys1 - keys2
print(len(difference12))
print(difference12)

11
{('other', 'could'), ('number', 'are there any'), ('number', 'why is the'), ('number', 'do you'), ('number', 'has'), ('number', 'does this'), ('number', 'are they'), ('yes/no', 'what is the man'), ('yes/no', 'what does the'), ('number', 'are'), ('number', 'do')}


In [14]:
difference21 = keys2 - keys1
print(len(difference21))
print(difference21)

11
{('number', 'is this person'), ('number', 'what is the man'), ('yes/no', 'what are'), ('number', 'what color is the'), ('yes/no', 'what are the'), ('number', 'are these'), ('number', 'is he'), ('yes/no', 'why is the'), ('number', 'is that a'), ('number', 'was'), ('yes/no', 'how')}


In [15]:
def sentence2words(sentence, map_person=True):
    def word_modifier(word):
        MAP = {'zero': '0', 'one': '1', 'two': '2', 'three': '3', 'four': '4', 'five': '5', 'six': '6',
               'seven': '7', 'eight': '8', 'nine': '9', 'ten': '10', 'both': '2',
               'inside': 'in', 'outside': 'out'}
        MAPPERSON = {'you': 'i', 'your': 'my', 'yours': 'mine', 'i': 'you', 'me': 'you',
                     'my': 'your', 'mine': 'yours'}
        if word in MAP:
            word = MAP[word]
        if map_person and word in MAPPERSON:
            word = MAPPERSON[word]
        return word
    question_words = ['what', 'where', 'why', 'how', 'when', 'who', 'whose', 'whom']
    words = re.split('[,. ?/_-]', sentence)
    words = [word_modifier(word.lower()) for word in words if word]
    words_no_short_verbs = []
    for word in words:
        if word[-2:] == "'s" and word[:-2] in question_words:
            words_no_short_verbs.append(word[:-2])
            words_no_short_verbs.append('is')
        else:
            words_no_short_verbs.append(word)
    return words_no_short_verbs

In [16]:
def preprocess_qa(q, a, question_type):
    question_type = sentence2words(question_type)
    nr_beginning_words = len(question_type)
    q = sentence2words(q)[nr_beginning_words:]
    a = sentence2words(a)
    return q, a

In [17]:
def find_word_index(words, word):
    for i, w in enumerate(words):
        if w == word:
            return i
    return -1

In [18]:
def find_or(words):
    return find_word_index(words, 'or')

In [19]:
def positional_matching(option, answer, is_first_option):
    matching_position = -1
    nr_matches = 0
    first_word = answer[0]
    for i, w in enumerate(option):
        if w == first_word:
            matching_position = i
            nr_matches = 1
            option = option[i+1:]
            answer = answer[1:]
            #print('o, a =', option, answer)
            for j, (w1, w2) in enumerate(zip(option, answer)):
                if w1 != w2:
                    break
                nr_matches += 1
                if not is_first_option:
                    matching_position += 1
            break
    return matching_position, nr_matches

In [20]:
def remove_duplicates(sentence):
    prev_word = sentence[0]
    new_sentence = [prev_word]
    for word in sentence[1:]:
        if word != prev_word:
            new_sentence.append(word)
            prev_word = word
    return new_sentence

In [21]:
def list2str(answer):
    answer = remove_duplicates(answer)
    answer[0] = answer[0].capitalize()
    answer = ' '.join(answer) + '.'
    answer = re.sub(r'\s([?.,!"](?:\s|$))', r'\1', answer)
    return answer

In [22]:
def find_number(words, direction):
    for i, w in enumerate(words[::direction]):
        if w.isnumeric():
            return i
    return -1

In [23]:
def split_or(q, a, question_type='', answer_type='other', verb_before_option='', or_position=-1):
    PREPOZITIONS = ['a', 'an', 'the', 'not']
    OFFSETLIMIT = 4
    is_numeric_answer = answer_type == 'number'
    verb_before_option = sentence2words(verb_before_option) if verb_before_option else []
    
    if question_type:
        q, a = preprocess_qa(q, a, question_type)
    
    if or_position == -1:
        or_position = find_or(q)
        if or_position == -1:
            print('Error. No "or"')
            return [None, None, None, None, None] # TODO Something custom, specific to the rule should be done
    
    option1 = q[:or_position]
    option2 = q[or_position+1:]
    position_option1, nr_matches1 = positional_matching(option1, a, True)
    position_option2, nr_matches2 = positional_matching(option2, a, False)
    
    if position_option1 > 1 and option1[position_option1-1] in PREPOZITIONS:
        position_option1 -= 1
    if position_option2 != -1 and position_option2 < len(option2) - 1 and option2[position_option2] in PREPOZITIONS:
        position_option2 += 1
    
    if position_option1 == -1:
        if position_option2 == -1:
            print('Error. Answer not found in options 1 and 2')
            return None, None, None, None
        else:
            choose_first_option = False
            length_option2 = position_option2 + 1
            position_option1 = max(len(option1) - length_option2, 0)
            ############################################################################################
            if is_numeric_answer and position_option2 > 0:
                number_distance = find_number(option1, -1)
                if number_distance != -1:
                    position_number = len(option1) - 1 - number_distance
                    if position_option1 <= OFFSETLIMIT:
                        position_option1 = min(position_number, position_option1)
                    if position_number != 0:
                        nr_w_before = min(position_number, position_option2)
                        orig_position_number = position_number
                        for w1, w2 in zip(option1[position_number-1::-1][:nr_w_before],
                                          option2[position_option2-1::-1][:nr_w_before]):
                            if w1 != w2:
                                break
                            position_number -= 1
                        if position_number != orig_position_number:
                            position_option1 = min(position_option1, position_number)
            #++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
            if is_numeric_answer and position_option2 < len(option2) - 1:
                number_distance = find_number(option1, -1)
                if number_distance != -1:
                    position_number = len(option1) - 1 - number_distance
                    if position_option1 <= OFFSETLIMIT:
                        position_option1 = min(position_number, position_option1)
                    nr_w_after = min(len(option1) - 1 - position_number, len(option2) - 1 - position_option2)
                    for w1, w2 in zip(option1[position_number+1:position_number+1+nr_w_after],
                                      option2[position_option2+1:position_option2+1+nr_w_after]):
                        if w1 != w2:
                            break
                        position_option2 += 1
            ############################################################################################
    else:
        if position_option2 == -1:
            choose_first_option = True
            length_option1 = len(option1) - position_option1
            position_option2 = max(length_option1 - 1, 0)
            ############################################################################################
            if is_numeric_answer and position_option1 > 0:
                number_distance = find_number(option2, 1)
                if number_distance != -1:
                    position_number = number_distance
                    if len(option2) - 1 - position_option2 <= OFFSETLIMIT:
                        position_option2 = max(position_number, position_option2)
                    nr_w_after = min(len(option1) - 1 - position_option1, len(option2) - 1 - position_option1)
                    orig_position_number = position_number
                    for w1, w2 in zip(option1[position_option1+1:position_option1+1+nr_w_after],
                                      option2[position_number+1:position_number+1+nr_w_after]):
                        if w1 != w2:
                            break
                        position_number += 1
                    if position_number != orig_position_number:
                        position_option2 = max(position_option2, position_number)
            #++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
            if is_numeric_answer and position_option1 < len(option1) - 1:
                number_distance = find_number(option2, 1)
                if number_distance != -1:
                    position_number = number_distance
                    if len(option2) - 1 - position_option2 <= OFFSETLIMIT:
                        position_option2 = max(position_number, position_option2)
                    else:
                        if position_number != 0:
                            nr_w_before = min(position_option1, position_number)
                            for w1, w2 in zip(option1[position_option1-1::-1][:nr_w_before],
                                              option2[position_number-1::-1][:nr_w_before]):
                                if w1 != w2:
                                    break
                                position_option1 -= 1
            ############################################################################################
        else:
            if nr_matches1 != nr_matches2:
                choose_first_option = nr_matches1 > nr_matches2
            else: # Choose the option with less words (Occam's razor)
                length_option2 = position_option2 + 1
                length_option1 = len(option1) - position_option1
                choose_first_option = length_option1 < length_option2
    
    if position_option1 > 1 and option1[position_option1-1] in PREPOZITIONS:
        position_option1 -= 1
    if position_option2 != -1 and position_option2 < len(option2) - 1 and option2[position_option2] in PREPOZITIONS:
        position_option2 += 1
        
    before_option1 = option1[:position_option1]
    option1 = option1[position_option1:]
    after_option2 = option2[position_option2+1:]
    option2 = option2[:position_option2+1]

    correct_option = option1 if choose_first_option else option2
    return before_option1 + verb_before_option + correct_option + after_option2

In [24]:
qq, aa = ('Are the sheep at least two legs mammals or at least four legs mammals with nice head?', '4')
split_or(qq, aa, 'are the', 'number')

['sheep', 'at', 'least', '4', 'legs', 'mammals', 'with', 'nice', 'head']

In [25]:
qq, aa = ('Are the sheep at least two legs mammals or at least four legs mammals with nice head?', '2')
split_or(qq, aa, 'are the', 'number')

['sheep', 'at', 'least', '2', 'legs', 'mammals', 'with', 'nice', 'head']

In [26]:
def default_answer(q, a, qtype, atype, verb='', predefined_answer=[]):
    or_position = find_or(q)
    if or_position != -1:
        return split_or(q, a, '', atype, verb)
    how_index = find_word_index(q, 'how')
    if how_index != -1 and how_index < len(q) - 1:
        word_after_how = q[how_index + 1]
        if word_after_how[:len('frequent')] in ['many', 'much', 'often', 'seldom', 'frequent', 'regular']:
            return q[:how_index] + a + q[how_index+2:]
    if predefined_answer:
        return predefined_answer
    return a + q

# Questions like "Is this man who I know ..." may be problematic because "know" is not the verb we are looking for
def yes_no_default_answer(q, a, qtype, atype, verb=[]):
    if verb:
        verb = [verb]
    if a == ['no']:
        verb = verb + ['not']
    pos_tags = [x[1] for x in nltk.pos_tag(q)]
    for possible_tag in ['VB', 'RB', 'JJ']: # e.g. 'is' can be before 'going' (VB), 'well' (RB), 'rich' (JJ) 
        for tag_position, tag in enumerate(pos_tags):
            if tag[:2] == possible_tag:
                return a + [','] + q[:tag_position] + verb + q[tag_position:]
    return a + [','] + verb + q    

def try_other_function(q, a, qtype, atype, possible_function_name, default_func=default_answer):
    q = q[1:]
    try:
        return eval(possible_function_name + '(q, a, qtype, atype)')
    except NameError:
        return default_func(q, a, qtype, atype)

def try_other_yes_no_function(q, a, qtype, atype, possible_function_name):
    q = q[1:]
    try:
        return eval(possible_function_name + '(q, a, qtype, atype)')
    except NameError:
        return default_answer(q, a, qtype, atype)

def number_are_the(q, a, qtype, atype):
    return default_answer(q, a, qtype, atype, verb = 'are')

def number_are_these(q, a, qtype, atype):
    return number_are(['these'], q, a, qtype, atype)

def number_are_there_any(q, a, qtype, atype):
    return number_are_there(q, a, qtype, atype)

def number_are_they(q, a, qtype, atype):
    return ['They', 'are'] + default_answer(q, a, qtype, atype)

def number_are(q, a, qtype, atype):
    #if q[0] == 'how':
    #    q.pop(0)
    return default_answer(q, a, qtype, atype, verb = 'are', predefined_answer = a + ['are'] + q)

def number_are_there(q, a, qtype, atype):
    return ['There', 'are'] + default_answer(q, a, qtype, atype)

def number_can_you(q, a, qtype, atype):
    return ['i', 'can'] + default_answer(q, a, qtype, atype, predefined_answer = q + [','] + a)

def number_do(q, a, qtype, atype):
    return default_answer(q, a, qtype, atype)

def number_do_you(q, a, qtype, atype):
    return default_answer(['i'] + q, a, qtype, atype)
 
def number_does_the(q, a, qtype, atype):
    return default_answer(['the'] + q, a, qtype, atype)

def number_does_this(q, a, qtype, atype):
    return default_answer(['this'] + q, a, qtype, atype)

def number_has(q, a, qtype, atype):
    return default_answer(q, a, qtype, atype, verb = 'has', predefined_answer = a + ['has'] + q)

def number_how(q, a, qtype, atype):
    if q[0] == 'many':
        return number_how_many(q[1:], a, qtype, atype)
    if q[0] == 'much':
        q = q[1:]
    return default_answer(q, a, qtype, atype)

def number_how_many(q, a, qtype, atype):
    return default_answer(q, a, qtype, atype)

def number_how_many_people_are(q, a, qtype, atype):
    return number_how_many(['people', 'are'] + q, a, qtype, atype)

def number_how_many_people_are_in(q, a, qtype, atype):
    return number_how_many(['people', 'are', 'in'] + q, a, qtype, atype)

def number_was(q, a, qtype, atype):
    return default_answer(q, a, qtype, atype, verb = 'was', predefined_answer = a + ['was'] + q)

def number_is(q, a, qtype, atype):
    return default_answer(q, a, qtype, atype, verb = 'is', predefined_answer = a + ['is'] + q)

def number_is_he(q, a, qtype, atype):
    return number_is(['he'] + q, a, qtype, atype)

def number_is_it(q, a, qtype, atype):
    return number_is(['it'] + q, a, qtype, atype)

def number_is_the(q, a, qtype, atype):
    return number_is(['the'] + q, a, qtype, atype)

def number_is_that(q, a, qtype, atype):
    return number_is(['that'] + q, a, qtype, atype)

def number_is_that_a(q, a, qtype, atype):
    return number_is(['that', 'a'] + q, a, qtype, atype)

def number_is_the_man(q, a, qtype, atype):
    return number_is(['the', 'man'] + q, a, qtype, atype)

def number_is_the_man(q, a, qtype, atype):
    return number_is(['the', 'person'] + q, a, qtype, atype)

def number_is_the_woman(q, a, qtype, atype):
    return number_is(['the', 'woman'] + q, a, qtype, atype)

def number_is_there(q, a, qtype, atype):
    return ['There', 'is'] + default_answer(q, a, qtype, atype)

def number_is_this(q, a, qtype, atype):
    return number_is(['this'] + q, a, qtype, atype)

def number_is_this_a(q, a, qtype, atype):
    return number_is(['this', 'a'] + q, a, qtype, atype)

def number_none_of_the_above(q, a, qtype, atype):
    possible_function_name = 'number_' + q[0]
    return try_other_function(q, a, qtype, atype, possible_function_name)

def number_what(q, a, qtype, atype):
    possible_function_name = 'number_what_' + q[0]
    return try_other_function(q, a, qtype, atype, possible_function_name)

def number_what_animal_is(q, a, qtype, atype):
    return default_answer(q, a, qtype, atype, verb = 'is', predefined_answer = a + ['is'] + q)

def number_what_are(q, a, qtype, atype):
    return default_answer(q, a, qtype, atype, verb = 'are', predefined_answer = a + ['are'] + q)

def number_what_are_the(q, a, qtype, atype):
    return number_what_are(['the'] + q, a, qtype, atype)

def number_what_brand(q, a, qtype, atype):
    is_index = find_word_index(q, 'is')
    return default_answer(q, a, qtype, atype, predefined_answer = a + q[is_index:])

def number_what_does_the(q, a, qtype, atype):
    return default_answer(q, a, qtype, atype, predefined_answer = ['The'] + q + a)

def number_what_is(q, a, qtype, atype):
    return default_answer(q, a, qtype, atype, verb = 'is', predefined_answer = a + ['is'] + q)

def number_what_is_on_the(q, a, qtype, atype):
    return number_what_is(['on', 'the'] + q, a, qtype, atype)

def number_what_is_the(q, a, qtype, atype):
    return number_what_is(['the'] + q, a, qtype, atype)

def number_what_is_the_man(q, a, qtype, atype):
    return number_what_is(['the', 'man'] + q, a, qtype, atype)

def number_what_color_is_the(q, a, qtype, atype):
    return number_what_is_the(q, a, qtype, atype)

def number_what_is_the_name(q, a, qtype, atype):
    return number_what_is(['the', 'name'] + q, a, qtype, atype)

def number_what_is_this(q, a, qtype, atype):
    return number_what_is(['this'] + q, a, qtype, atype)

def number_what_kind_of(q, a, qtype, atype):
    return default_answer(q, a, qtype, atype)

def number_what_number_is(q, a, qtype, atype):
    predefined_answer = q + ['is', 'number'] + a if q[0] == 'the' else a + ['is'] + q
    return default_answer(q, a, qtype, atype, verb = 'is number', predefined_answer=predefined_answer)

def number_what_time(q, a, qtype, atype):
    clock_index = find_word_index(q, 'clock')
    if q[-2] == 'is' and q[-1] == 'it' or clock_index != -1:
        return ['It', 'is'] + a
    return default_answer(q, a, qtype, atype, verb = 'is', predefined_answer = a + ['is'] + q)

def number_what_type_of(q, a, qtype, atype):
    return default_answer(q, a, qtype, atype)

def number_where_is_the(q, a, qtype, atype):
    q = ['the'] + q
    if q[-1] == 'from':
        return q[:-1] + ['is', 'from'] + a
    return default_answer(q, a, qtype, atype, verb = 'is', predefined_answer = q + ['is'] + a)
    
def number_which(q, a, qtype, atype):
    return default_answer(q, a, qtype, atype, a + q)

def number_who_is(q, a, qtype, atype):
    return default_answer(q, a, qtype, atype, verb = 'is', predefined_answer = a + ['is'] + q)

def number_why(q, a, qtype, atype):
    return default_answer(q, a, qtype, atype, predefined_answer = ['Because'] + a)

def number_why_is_the(q, a, qtype, atype):
    return default_answer(q, a, qtype, atype, verb = 'is', predefined_answer = a + ['is', 'the'] + q)
    
def other_are(q, a, qtype, atype):
    return default_answer(q, a, qtype, atype, verb = 'are', predefined_answer = q + ['are'] + a)

def other_are_the(q, a, qtype, atype):
    return other_are(['the'] + q, a, qtype, atype)

def other_are_there(q, a, qtype, atype):
    return other_are(['there'] + q, a, qtype, atype)
    
def other_are_these(q, a, qtype, atype):
    return other_are(['these'] + q, a, qtype, atype)

def other_are_they(q, a, qtype, atype):
    return other_are(['they'] + q, a, qtype, atype)

def other_can_you(q, a, qtype, atype):
    return ['I', 'can'] + default_answer(q, a, qtype, atype, predefined_answer = q + a)

def other_could(q, a, qtype, atype):
    return default_answer(q, a, qtype, atype)

def other_do(q, a, qtype, atype):
    return default_answer(q, a, qtype, atype)

def other_do_you(q, a, qtype, atype):
    return other_do(['i'] + q, a, qtype, atype)

def other_does_the(q, a, qtype, atype):
    return other_do(['the'] + q, a, qtype, atype)
    
def other_does_this(q, a, qtype, atype):
    return other_do(['this'] + q, a, qtype, atype)    

def other_has(q, a, qtype, atype):
    return default_answer(q, a, qtype, atype, verb = 'has', predefined_answer = q + ['has'] + a)

def other_how(q, a, qtype, atype):
    possible_function_name = 'other_how_' + q[0]
    return try_other_function(q, a, qtype, atype, possible_function_name)

def other_how_many(q, a, qtype, atype):
    return default_answer(q, a, qtype, atype)

def other_how_many_people_are(q, a, qtype, atype):
    return other_how_many(['people', 'are'] + q, a, qtype, atype)

def other_how_many_people_are_in(q, a, qtype, atype):
    return other_how_many(['people', 'are', 'in'] + q, a, qtype, atype)

def other_is(q, a, qtype, atype):
    return default_answer(q, a, qtype, atype, verb = 'is', predefined_answer = q + ['is'] + a)

def other_is_he(q, a, qtype, atype):
    return other_is(['he'] + q, a, qtype, atype)

def other_is_it(q, a, qtype, atype):
    return other_is(['it'] + q, a, qtype, atype)

def other_is_that_a(q, a, qtype, atype):
    return other_is(['that', 'a'] + q, a, qtype, atype)

def other_is_the(q, a, qtype, atype):
    return other_is(['the'] + q, a, qtype, atype)

def other_is_the_man(q, a, qtype, atype):
    return other_is(['the', 'man'] + q, a, qtype, atype)

def other_is_the_person(q, a, qtype, atype):
    return other_is(['the', 'person'] + q, a, qtype, atype)

def other_is_the_woman(q, a, qtype, atype):
    return other_is(['the', 'woman'] + q, a, qtype, atype)

def other_is_there(q, a, qtype, atype):
    return other_is(['there'] + q, a, qtype, atype)

def other_is_there_a(q, a, qtype, atype):
    return other_is(['there', 'a'] + q, a, qtype, atype)

def other_is_this(q, a, qtype, atype):
    return other_is(['this'] + q, a, qtype, atype)

def other_is_this_a(q, a, qtype, atype):
    return other_is(['this', 'a'] + q, a, qtype, atype)

def other_is_this_an(q, a, qtype, atype):
    return other_is(['this', 'an'] + q, a, qtype, atype)

def other_is_this_person(q, a, qtype, atype):
    return other_is(['this', 'person'] + q, a, qtype, atype)

def other_none_of_the_above(q, a, qtype, atype):
    possible_function_name = 'other_' + q[0]
    return try_other_function(q, a, qtype, atype, possible_function_name)

def other_was(q, a, qtype, atype):
    return default_answer(q, a, qtype, atype, verb = 'was', predefined_answer = q + ['was'] + a)

def other_what(q, a, qtype, atype):
    possible_function_name = 'other_what_' + q[0]
    return try_other_function(q, a, qtype, atype, possible_function_name)

def other_what_animal_is(q, a, qtype, atype):
    return default_answer(q, a, qtype, atype, verb = 'is', predefined_answer = q + ['is'] + a)

def other_what_are(q, a, qtype, atype):
    return default_answer(q, a, qtype, atype, verb = 'are', predefined_answer = q + ['are'] + a)

def other_what_are_the(q, a, qtype, atype):
    return other_what_are(['the'] + q, a, qtype, atype)

def other_what_brand(q, a, qtype, atype):
    return default_answer(q, a, qtype, atype)

def other_what_color(q, a, qtype, atype):
    return default_answer(q, a, qtype, atype)

def other_what_color_are(q, a, qtype, atype):
    return default_answer(q, a, qtype, atype, verb = 'are', predefined_answer = q + ['are'] + a)

def other_what_color_are_the(q, a, qtype, atype):
    return other_what_color_are(['the'] + q, a, qtype, atype)

def other_what_color_is(q, a, qtype, atype):
    return default_answer(q, a, qtype, atype, verb = 'is', predefined_answer = q + ['is'] + a)

def other_what_color_is_the(q, a, qtype, atype):
    return default_answer(['the'] + q, a, qtype, atype, verb = 'is', predefined_answer = q + ['is'] + a)

def other_what_does_the(q, a, qtype, atype):
    return default_answer(['the'] + q, a, qtype, atype)

def other_what_is(q, a, qtype, atype):
    return default_answer(q, a, qtype, atype, verb = 'is', predefined_answer = q + ['is'] + a)

def other_what_is_in_the(q, a, qtype, atype):
    return other_what_is(['in', 'the'] + q, a, qtype, atype)

def other_what_is_on_the(q, a, qtype, atype):
    return other_what_is(['on', 'the'] + q, a, qtype, atype)

def other_what_is_the(q, a, qtype, atype):
    return other_what_is(['the'] + q, a, qtype, atype)

def other_what_is_the_color_of_the(q, a, qtype, atype):
    return other_what_is(['the', 'color', 'of', 'the'] + q, a, qtype, atype)

def other_what_is_the_man(q, a, qtype, atype):
    return other_what_is(['the', 'man'] + q, a, qtype, atype)

def other_what_is_the_name(q, a, qtype, atype):
    return other_what_is(['the', 'name'] + q, a, qtype, atype)

def other_what_is_the_person(q, a, qtype, atype):
    return other_what_is(['the', 'person'] + q, a, qtype, atype)

def other_what_is_the_woman(q, a, qtype, atype):
    return other_what_is(['the', 'woman'] + q, a, qtype, atype)

def other_what_is_this(q, a, qtype, atype):
    return other_what_is(['this'] + q, a, qtype, atype)

def other_what_kind_of(q, a, qtype, atype):
    return default_answer(q, a, qtype, atype)

def other_what_number_is(q, a, qtype, atype):
    return default_answer(q, a, qtype, atype, verb = 'number is', predefined_answer = q + ['number', 'is'] + a)

def other_what_room_is(q, a, qtype, atype):
    return default_answer(q, a, qtype, atype, verb = 'room is', predefined_answer = q + ['room', 'is'] + a)

def other_what_sport_is(q, a, qtype, atype):
    return default_answer(q, a, qtype, atype, verb = 'sport is', predefined_answer = q + ['sport', 'is'] + a)

def other_what_time(q, a, qtype, atype):
    return other_what(q, a, qtype, atype)

def other_what_type_of(q, a, qtype, atype):
    return other_what_kind_of(q, a, qtype, atype)

def other_where_is(q, a, qtype, atype):
    return default_answer(q, a, qtype, atype, verb = 'is', predefined_answer = q + ['is'] + a)

def other_where_is_the(q, a, qtype, atype):
    return other_where_is(['the'] + q, a, qtype, atype)

def other_where_are(q, a, qtype, atype):
    return default_answer(q, a, qtype, atype, verb = 'are', predefined_answer = q + ['are'] + a)

def other_where_are_the(q, a, qtype, atype):
    return other_where_are(['the'] + q, a, qtype, atype)

def other_which(q, a, qtype, atype):
    return default_answer(q, a, qtype, atype)

def other_who_is(q, a, qtype, atype):
    return default_answer(q, a, qtype, atype, verb = 'is', predefined_answer = a + ['is'] + q)

def other_why(q, a, qtype, atype):
    if q[0] == 'is':
        return other_why_is(q[1:], a, qtype, atype)
    return default_answer(q, a, qtype, atype, ['Because'] + a + q)

def other_why_is(q, a, qtype, atype):
    return default_answer(q, a, qtype, atype, verb = 'is', predefined_answer = q + ['is', 'because'] + a)

def other_why_is_the(q, a, qtype, atype):
    return other_why_is(['the'] + q, a, qtype, atype)

def yes_no_are(q, a, qtype, atype):
    return yes_no_default_answer(q, a, qtype, atype, 'are')

def yes_no_are_the(q, a, qtype, atype):
    return yes_no_are(['the'] + q, a, qtype, atype)

def yes_no_are_there(q, a, qtype, atype):
    not_term = ['not'] if a == ['no'] else []
    return a + [',', 'there', 'are'] + not_term + q

def yes_no_are_there_any(q, a, qtype, atype):
    not_term = ['no'] if a == ['no'] else ['some']
    return a + [',', 'there', 'are'] + not_term + q

def yes_no_are_these(q, a, qtype, atype):
    return yes_no_are(['these'] + q, a, qtype, atype)

def yes_no_are_they(q, a, qtype, atype):
    return yes_no_are(['they'] + q, a, qtype, atype)

def yes_no_can(q, a, qtype, atype):
    return yes_no_default_answer(q, a, qtype, atype, 'can')

def yes_no_can_you(q, a, qtype, atype):
    return yes_no_can(['i'] + q, a, qtype, atype)

def yes_no_could(q, a, qtype, atype):
    return yes_no_default_answer(q, a, qtype, atype, 'could')

def yes_no_do(q, a, qtype, atype):
    return yes_no_default_answer(q, a, qtype, atype)

def yes_no_do_you(q, a, qtype, atype):
    return yes_no_do(['i'] + q, a, qtype, atype)

def yes_no_does_the(q, a, qtype, atype):
    return yes_no_do(['the'] + q, a, qtype, atype)

def yes_no_does_this(q, a, qtype, atype):
    return yes_no_do(['this'] + q, a, qtype, atype)

def yes_no_has(q, a, qtype, atype):
    return yes_no_default_answer(q, a, qtype, atype, 'has')

def yes_no_is(q, a, qtype, atype):
    return yes_no_default_answer(q, a, qtype, atype, 'is')

def yes_no_is_he(q, a, qtype, atype):
    return yes_no_is(['he'] + q, a, qtype, atype)

def yes_no_is_it(q, a, qtype, atype):
    return yes_no_is(['it'] + q, a, qtype, atype)

def yes_no_is_that_a(q, a, qtype, atype):
    return yes_no_is(['that', 'a'] + q, a, qtype, atype)

def yes_no_is_the(q, a, qtype, atype):
    return yes_no_is(['the'] + q, a, qtype, atype)

def yes_no_is_the_man(q, a, qtype, atype):
    return yes_no_is(['the', 'man'] + q, a, qtype, atype)

def yes_no_is_the_person(q, a, qtype, atype):
    return yes_no_is(['the', 'person'] + q, a, qtype, atype)

def yes_no_is_the_woman(q, a, qtype, atype):
    return yes_no_is(['the', 'woman'] + q, a, qtype, atype)

def yes_no_is_there(q, a, qtype, atype):
    not_term = ['not'] if a == ['no'] else []
    return a + [',', 'there', 'is'] + not_term + q

def yes_no_is_there_a(q, a, qtype, atype):
    return yes_no_is_there(['a'] + q, a, qtype, atype)

def yes_no_is_this(q, a, qtype, atype):
    return yes_no_is(['this'] + q, a, qtype, atype)

def yes_no_is_this_a(q, a, qtype, atype):
    not_term = ['not'] if a == ['no'] else []
    return a + [',', 'this', 'is'] + not_term + ['a'] + q

def yes_no_is_this_an(q, a, qtype, atype):
    not_term = ['not'] if a == ['no'] else []
    return a + [',', 'this', 'is'] + not_term + ['an'] + q

def yes_no_is_this_person(q, a, qtype, atype):
    return yes_no_is(['this', 'person'] + q, a, qtype, atype)

def yes_no_none_of_the_above(q, a, qtype, atype):
    possible_function_name = 'yes_no_' + q[0]
    return try_other_function(q, a, qtype, atype, possible_function_name, yes_no_default_answer)

def yes_no_was(q, a, qtype, atype):
    return yes_no_default_answer(q, a, qtype, atype, 'was')

def yes_no_what(q, a, qtype, atype):
    possible_function_name = 'yes_no_' + q[0]
    return try_other_function(q, a, qtype, atype, possible_function_name, yes_no_default_answer)

def yes_no_what_does_the(q, a, qtype, atype):
    return yes_no_does_the(q, a, qtype, atype)

def yes_no_what_is(q, a, qtype, atype):
    return yes_no_is(q, a, qtype, atype)

def yes_no_what_is_the(q, a, qtype, atype):
    return yes_no_is_the(q, a, qtype, atype)

def yes_no_what_is_the_man(q, a, qtype, atype):
    return yes_no_is_the_man(q, a, qtype, atype)

def yes_no_what_color_is_the(q, a, qtype, atype):
    return yes_no_what_is_the(q, a, qtype, atype)

def yes_no_what_are(q, a, qtype, atype):
    return yes_no_are(q, a, qtype, atype)

def yes_no_what_are_the(q, a, qtype, atype):
    return yes_no_are_the(q, a, qtype, atype)

def yes_no_why(q, a, qtype, atype):
    possible_function_name = 'yes_no_' + q[0]
    return try_other_function(q, a, qtype, atype, possible_function_name, yes_no_default_answer)

def yes_no_why_is(q, a, qtype, atype):
    return yes_no_default_answer(q, a, qtype, atype, 'is')

def other_why_is_the(q, a, qtype, atype):
    return yes_no_why_is(['the'] + q, a, qtype, atype)

def yes_no_would(q, a, qtype, atype):
    return yes_no_default_answer(q, a, qtype, atype, 'would')

def yes_no_will(q, a, qtype, atype):
    return yes_no_default_answer(q, a, qtype, atype, 'will')

def yes_no_how(q, a, qtype, atype):
    possible_function_name = 'yes_no_' + q[0]
    return try_other_function(q, a, qtype, atype, possible_function_name, yes_no_default_answer)

In [27]:
for (atype, qtype), vals in dic.items():
    q, a = vals[0]
    atype = 'yes/no' if a == 'yes' or a == 'no' else atype
    atype = 'number' if a.isnumeric() else atype
    func = '_'.join(sentence2words(atype + ' ' + qtype, map_person=False))
    print('q =', q)
    print('a =', a)
    print('qtype =', qtype)
    print('atype =', atype)
    qtype = '' if qtype == 'none of the above' else qtype
    q, a = preprocess_qa(q, a, qtype)
    #print(q)
    #print(a)
    #print(func)
    result = eval(func + '(q, a, qtype, atype)')
    #print('result =', result)
    print('result =', list2str(result))
    print('*' * 100)
    # TODO check if after "1" there is a NN ending in "s" and also before/after "1" if there is "are/have/etc."
    # Would this be helpful though?

q = Are many people playing tennis?
a = yes
qtype = are
atype = yes/no
result = Yes, many people are playing tennis.
****************************************************************************************************
q = Are the sheep two legs mammals or four legs mammals?
a = 4
qtype = are the
atype = number
result = Sheep are 4 legs mammals.
****************************************************************************************************
q = Are there 1 or 2 ovens?
a = 2
qtype = are there
atype = number
result = There are 2 ovens.
****************************************************************************************************
q = Are there any people in the image?
a = 1
qtype = are there any
atype = number
result = There are 1 people in the image.
****************************************************************************************************
q = Are they playing with one ball or two?
a = 1
qtype = are they
atype = number
result = They are playing with 1 ball.
***********

In [28]:
qq = ['trees', 'line', '2', 'sides', 'of', 'the', 'street', 'or', 'only', '1', 'side']
aa = ['1', 'side']
split_or(qq, aa, '', 'number', '')

['trees', 'line', 'only', '1', 'side']

In [29]:
['trees', 'line', 'only', '1', 'side']

['trees', 'line', 'only', '1', 'side']